In [1]:
import numpy as np
import pandas as pd
import cv2
import pickle
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from keras.models import Model, load_model, Sequential

Using TensorFlow backend.


In [3]:
vector_size = 100
batch_size = 10
epochs = 150
frames_in_each_sample = 66
frame_shape = (75, 175)  # width, height
channels = 3
with open('Vectorizor.pkl', 'rb') as file:
    vect = pickle.load(file)

In [4]:
def preprocessing(video_path):
    """Get Video from video_path"""
    cap = cv2.VideoCapture(video_path)
    vid = []
    while True:
        ret, img = cap.read()
        if not ret:
            break
        vid.append(cv2.resize(img, frame_shape, cv2.INTER_AREA))

    vid = np.array(vid, dtype=np.float32).reshape(-1,
                                                  frame_shape[1],
                                                  frame_shape[0],
                                                  channels)
    new_vid = vid[:frames_in_each_sample]
    print(new_vid.shape)
    return new_vid


In [5]:
model = load_model('video_captioning_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 66, 175, 75, 64)   5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 66, 87, 37, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 66, 87, 37, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 33, 43, 18, 128)   0         
_________________________________________________________________
conv3a (Conv3D)              (None, 33, 43, 18, 256)   884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 33, 43, 18, 256)   1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 16, 21, 9, 256)    0         
__________

In [6]:
test_vid = './test_clips/FRIENDS_WITH_BENEFITS_DVS101.avi'
new_vid =  preprocessing(test_vid)
new_vid = np.array([new_vid])
print(new_vid.shape)
print type(new_vid)

(66, 175, 75, 3)
(1, 66, 175, 75, 3)
<type 'numpy.ndarray'>


In [7]:
temp = model.predict(new_vid, batch_size=1, verbose=0)

In [8]:
cap = vect.inverse_transform(temp)
print ("Predicted Caption:", str(' '.join(cap[0])))

('Predicted Caption:', 'approaches arm arms away bed black boy brow car chest climbs dark door drops enters eyes face faces father feet finds floor follows forward gaze gazes gets gives glances glass grabs ground gun hair haired hand hands head heads holds house inside kisses later leans leaves lies lifts light lips look looks lowers man men mouth nods open opens outside past peers phone picks pulls puts reaches reads red room runs seat sets shakes shifts shows sits smile smiles stands stares steps stops street table takes turns view walks wall watch watches water wearing white window woman young')


In [9]:
temp.shape

(1, 100)